# R&D Trading Strategy
## Rudraksh Garg

### ** All final results are at the bottom of the notebook. **

## Imports

Data processing libraries: 

- pandas
- numpy

Datetime calculations: 
- dateutil

Runtime Output: 
- warnings

Regressions: 
- statsmodel

Sharpe Ratio Calculation: 
- math


In [33]:
# Imports
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import warnings
import statsmodels.api as sm
import math

## Data Functions

Below are functions that are used to gather, clean, and process data as inputs for backtesting. There are functions that process the monthly stock returns and other functions that process company R&D expenses. This section of functions also calculate the variables that will be used to create the quintiles that will be used to construct the long-short portfolio and calculate the effectiveness of the strategy.

In [23]:
def get_stocks_data(file = "data/stocks.csv"):
    '''
    Get stock monthly return and shares outstanding
        Parameters:
                file (string): file path

        Returns:
                (DateFrame): stock data 
    '''
    return pd.read_csv(file)

def clean_stock_data(stocks):
    '''
    Prepares stock data
        Parameters:
                stocks (DateFrame): stock data

        Returns:
                (DateFrame): cleaned stock data 
    '''
    # convert to datetime
    stocks["datadate"] = pd.to_datetime(stocks["datadate"])
    
    # Quarterly to monthly number of shares outstanding
    stocks = stocks.sort_values(by=['LPERMNO', "datadate"])
    stocks["cshoq"] = stocks.groupby("tic")["cshoq"].fillna(method='ffill')
    
    # Remove 6000-6999 sic
    stocks = stocks[(stocks["sic"] <= 6000) | (stocks["sic"] > 6999)]
    
    # Keep exchange code 11-19
    stocks = stocks[(stocks["exchg"] >= 11) & (stocks["exchg"] <= 19)]

    # Remove unreasonably high returns
    stocks = stocks[(stocks["trt1m"] < 500)]

    # Remove impossible returns
    stocks = stocks[(stocks["trt1m"] > -100)]

    # Remove stocks that have 0 or negative market cap
    stocks = stocks[(stocks["cshoq"] > 0)]

    # If price is missing, make it nan
    stocks["prccm"] = stocks["prccm"].replace({0:np.nan})

    # Remove records that have misssing shares outstanding or price
    stocks = stocks.dropna(subset=["cshoq", "prccm"])

    # Convert Market Cap out of millions
    stocks["Market Cap"] = stocks["prccm"] * stocks["cshoq"] * 1000000

    # Ensure no misssing data
    assert stocks.isna().sum().any() == False

    # Get desired columns
    stocks = stocks[["datadate", "GVKEY", "LPERMNO", "Market Cap", "trt1m"]]

    return stocks

def save_gvkeys(stocks, column = "GVKEY"):
    '''
    Writes all gvkeys to file in format WRDS can read to pull fundamentals 
        Parameters:
                stocks (DateFrame): stock data
                column (string): column of gvkeys

    '''
    with open('gvkeys.txt', mode='wt', encoding='utf-8') as myfile:
        myfile.write("\n".join([str(i) for i in stocks[column].unique().tolist()]))


def get_fundamentals(file = "data/fundamentals.csv"):
    '''
    Get annual company fundamentals and their R&D Spending
        Parameters:
                file (string): file path

        Returns:
                (DateFrame): fundamentals data 
    '''
    return pd.read_csv(file)[["datadate","GVKEY", "LPERMNO", "xrd"]]

def clean_fundamentals(fundamentals):
    '''
    Prepares fundamentals data
        Parameters:
                fundamentals (DateFrame): fundamentals data

        Returns:
                (DateFrame): cleaned fundamentals data 
    '''
    fundamentals["datadate"] = pd.to_datetime(fundamentals["datadate"])
    fundamentals["xrd"] = fundamentals["xrd"].replace({np.nan:0})
    return fundamentals

def split_firms(fundamentals, debug = False):
    '''
    Separates rd firms and non-rd firms in 2 dataframes
        Parameters:
                fundamentals (DateFrame): fundamentals data
                debug (boolean): print out debug statements of percent complete

        Returns:
                (DateFrame): all rd firms across time
                (DateFrame): all non rd firms across time
    '''
    # Find continuous time period sets
    ser_date = pd.to_datetime(fundamentals['datadate'])

    # Find continuous rows that refer to the same GVKEY without breaks in time by years
    ser_group = ((((ser_date.shift() + pd.DateOffset(years=1)) != ser_date) | 
                (fundamentals['GVKEY'].ne(fundamentals['GVKEY'].shift().bfill()).astype(int) != 0))
                .cumsum())

    g_df = fundamentals.groupby(ser_group) 

    # Get the list of continuous rows for each gvkey
    df_periods = (g_df['GVKEY','datadate'].first() 
            .join(g_df['datadate'].last(),rsuffix='_') 
            .rename(columns={'datadate':'start','datadate_':'end'})) 

    # Find the length of the continuous time periods
    df_periods["Period Length"] = df_periods["end"] - df_periods["start"]
    df_periods["Period Length"] = df_periods["Period Length"] / np.timedelta64(1, "Y")

    # Fine rows, dates, gvkey sets that have less than 5 years
    drop_periods = df_periods[df_periods["Period Length"] < 4.999].reset_index(drop=True)

    # Drop all indices that do not fall in a 5 year period
    drop_idxs = []
    num_rows = len(drop_periods)
    for i, row, in drop_periods.iterrows():
        gvkey = row["GVKEY"]
        start = row["start"]
        end = row["end"]

        drop_idxs = drop_idxs + list(fundamentals[(fundamentals["GVKEY"] == gvkey) & (fundamentals["datadate"] >= start) & (fundamentals["datadate"] <= end)].index.values) #rd_firms
        
        if debug:
            if i % 1000 == 0:
                print(i/num_rows)

    # Keep rows that have 5 or more years of R&D expenses
    rd_firms = fundamentals.drop(drop_idxs)
    # Rows that have less than 5 years of R&D expenses are non rd firms
    non_rd_firms = fundamentals[fundamentals.index.isin(drop_idxs)]

    # Find firms and dates that firm has not invested in RD for 5 years
    temp = rd_firms.groupby('GVKEY')["xrd"].rolling(5).sum() == 0
    idxs = [i[1] for i in temp[temp].index]

    # Ensure that all indices of 0 RD for 5 years are true indices
    assert list(set(idxs) - set(list(rd_firms.index))) == []

    # Add these 0 RD firms to the non rd firms data and remove from rd firms data
    more_non_rd_firms = rd_firms.loc[idxs].sort_values("GVKEY")
    non_rd_firms = pd.concat([non_rd_firms, more_non_rd_firms])
    rd_firms = rd_firms[~rd_firms.index.isin(idxs)]

    # Convert RD expense out of millions
    rd_firms["xrd"] = rd_firms["xrd"]  * 1000000

    # NOTE: time shift 3 months because of delays in 10-K release
    non_rd_firms["datadate"] = non_rd_firms["datadate"] + pd.Timedelta(days=90)
    rd_firms["datadate"] = rd_firms["datadate"] + pd.Timedelta(days=90)

    return rd_firms, non_rd_firms

def calculate_rd_capital(rd_firms):
    '''
    Creates a column in rd firms data that reflects rd capital spend over 5 years
        Parameters:
                rd_firms (DateFrame): rd firms data

        Returns:
                (DateFrame): cleaned fundamentals data 
    '''
    # RDC_t = 0.2(XRD_t-4) + 0.4(XRD_t-4) + 0.6(XRD_t-2) + 0.8(XRD_t-1) + XRD_t
    weights = np.array([0.2, 0.4, 0.6, 0.8, 1.0])

    rd_firms["R&D Capital"] = rd_firms.groupby('GVKEY')["xrd"].rolling(len(weights)).apply(lambda x: np.sum(weights*x)).reset_index()[["level_1", "xrd"]].set_index("level_1")
    rd_firms.dropna(subset="R&D Capital", inplace=True)
    return rd_firms


def get_market_cap(rd_firms, non_rd_firms, stocks):
    '''
    Merges monthly market cap information with annual fundamentals
        Parameters:
                rd_firms (DateFrame): rd firms data across time
                non_rd_firms (DateFrame): non rd firms data across time
                stocks (DataFrame): cleaned stocks data 

        Returns:
                (DateFrame): all rd firms across time with most recent market cap
                (DateFrame): all non rd firms across time with most recent market cap
    '''
    # For later splitting
    rd_firms["type"] = "rd"
    non_rd_firms["type"] = "non-rd"

    # Combine for one look loop
    firms = pd.concat([rd_firms, non_rd_firms])

    # Get most recent market cap for each firm at the time of their 10-k release
    market_caps_dfs = []
    #NOTE: When using LPERMNO, this merge takes over an hour....results are similar when using GVKEY
    for GVKEY in firms["GVKEY"].unique():
        firm = firms[firms["GVKEY"] == GVKEY].sort_values("datadate")
        stock_firm = stocks[stocks["GVKEY"] == GVKEY].sort_values("datadate")

        # If market cap not available for current date, just use the most previous month market cap that has market cap data
        mark_cap = pd.merge_asof(firm, stock_firm[["datadate", "Market Cap"]], on="datadate", allow_exact_matches=True, direction="backward")
        market_caps_dfs.append(mark_cap)
    
    firms = pd.concat(market_caps_dfs)

    # Split type
    rd_firms = firms[firms["type"] == 'rd'].drop("type", axis=1)
    non_rd_firms = firms[firms["type"] == 'non-rd'].drop("type", axis=1)

    return rd_firms, non_rd_firms

def calculate_sorting_var(rd_firms):
    '''
    Create sorting column for rd firms that use rd capital and market cap
        Parameters:
                rd_firms (DateFrame): rd firms data across time with market cap

        Returns:
                (DateFrame): rd firms with sorting variable column
    '''
    # Normalize on market cap
    rd_firms["R&D Capital/Market Cap"] = rd_firms["R&D Capital"] / rd_firms["Market Cap"]
    return rd_firms


def get_most_recent_report(rd_firms, non_rd_firms, reconst_month = 4, reconst_day = 1):
    '''
    Use most recent fundamentals report in a year if there is more than one
        Parameters:
                rd_firms (DateFrame): rd firms data across time
                non_rd_firms (DateFrame): non rd firms data across time
                reconst_month (int): month number that portfolio will be reconstituted
                reconst_day (int): day number that portfolio will be reconstituted

        Returns:
                (DateFrame): all rd firms across time with most recent 10-K report
                (DateFrame): all non rd firms across time with most recent 10-K report
    '''
    # Get the year that will use this sorting variable
    rd_firms["year"] = [i.year+1 if i >= pd.Timestamp(year=i.year, month=reconst_month, day=reconst_day) else i.year for i in rd_firms["datadate"]]
    non_rd_firms["year"] = [i.year+1 if i >= pd.Timestamp(year=i.year, month=reconst_month, day=reconst_day) else i.year for i in non_rd_firms["datadate"]]
    
    # Get the most recent by keeping last
    rd_firms = rd_firms.drop_duplicates(["year", "GVKEY"], keep="last")
    non_rd_firms = non_rd_firms.drop_duplicates(["year", "GVKEY"], keep="last")
    
    # Confirm that there is only one report per year for each firm
    for year in rd_firms["year"].unique():
        assert (rd_firms[rd_firms["year"] == year]["GVKEY"].value_counts() == 1).all() == True
        assert (non_rd_firms[non_rd_firms["year"] == year]["GVKEY"].value_counts() == 1).all() == True

    return rd_firms, non_rd_firms

def remove_sorting_var_outliers(rd_firms, threshold = .99):
    '''
    Remove unreasonably high sorting var where the capital invested is much higher than the market cap
        Parameters:
                rd_firms (DateFrame): rd firms data across time with sorting variable

        Returns:
                (DateFrame): all rd firms across time with cleaned outliers
    '''
    rd_firms = rd_firms[rd_firms["R&D Capital/Market Cap"] < rd_firms["R&D Capital/Market Cap"].quantile(threshold)]
    return rd_firms

## Backtest and Get Portfolio Returns

Below are functions that are used to bin stocks based on the sorting variable, create the equal weighted and value weighted portfolio. There are also functions that process the risk free rate data and use that in order to calculate excess return of the portfolio. 

In [36]:
def get_bins(rd_firms, non_rd_firms, current_year = 1981, max_year=2022, month_reconst=4, day_reconst=1):
    '''
    Bin the sorting variable into 5 buckets from lowest to highest
        Parameters:
                rd_firms (DateFrame): rd firms data across time with sorting variable
                non_rd_firms (DateFrame): rd firms data across time with sorting variable
                current_year (int): earliest year of backtest
                max_year (int): most recent year of backtest
                reconst_month (int): month number that portfolio will be reconstituted
                reconst_day (int): day number that portfolio will be reconstituted

        Returns:
                (DateFrame): data for each year and it's respective companies in each bin
    '''

    dates = []

    H = []
    four = []
    three = []
    two = []
    L = []
    non_rd = []

    number_bins = 5
    labels = ["L", "2", "3", "4", "H"]

    # Iterate through each year
    while current_year <= max_year:
        
        # Get all rd and non_rd firms of that year
        current_reconst_firms = rd_firms[rd_firms["year"] == current_year]
        current_reconst_firms_non_rd = non_rd_firms[non_rd_firms["year"] == current_year]


        
        
        switch = False
        
        # If there are rd firms that year
        if current_reconst_firms.shape[0] > 0:
            # Bin firms
            current_reconst_firms["R&D Bins"] = pd.qcut(current_reconst_firms["R&D Capital/Market Cap"], number_bins, labels=labels)
            # Add to lists for columns
            H.append(current_reconst_firms[current_reconst_firms["R&D Bins"]=="H"]["GVKEY"].values)
            four.append(current_reconst_firms[current_reconst_firms["R&D Bins"]=="4"]["GVKEY"].values)
            three.append(current_reconst_firms[current_reconst_firms["R&D Bins"]=="3"]["GVKEY"].values)
            two.append(current_reconst_firms[current_reconst_firms["R&D Bins"]=="2"]["GVKEY"].values)
            L.append(current_reconst_firms[current_reconst_firms["R&D Bins"]=="L"]["GVKEY"].values)
            switch = True
            dates.append(pd.Timestamp(year=current_year, month=month_reconst, day=day_reconst))
        
            # Do the same for non rd firms
            if current_reconst_firms_non_rd.shape[0] > 0:
                non_rd.append(current_reconst_firms_non_rd["GVKEY"].values)
                


        current_year += 1

    # Create table
    tic_bins = pd.DataFrame()

    tic_bins["date"] = dates
    tic_bins["L"] = L
    tic_bins["2"] = two
    tic_bins["3"] = three
    tic_bins["4"] = four
    tic_bins["H"] = H
    tic_bins["Non R&D"] = non_rd

    return tic_bins

def get_rf_rate_monthly(stocks, file = "data/F-F_Research_Data_Factors_monthly.CSV"):
    '''
    Create a series of the risk free rate every month
        Parameters:
                stocks (DateFrame): all stock data to get dates
                file (DateFrame): file path to risk free data
              

        Returns:
                (Series): risk free rate each month in past
    '''

    # Pull the Fama French data which has the risk free rate per month in it
    rf = pd.read_csv(file).rename({"Unnamed: 0": "datadate"}, axis=1)[["datadate", "RF"]]

    # Convert to datetime
    rf["datadate"] = pd.to_datetime(rf["datadate"], format="%Y%m")
    
    # Remove copies and any missing months
    rf.drop_duplicates(subset="datadate", inplace=True, keep="last")
    rf.dropna(inplace=True)
    
    # Get most recent rf for a month if it is not available for all months in analysis
    rf = rf.sort_values("datadate")
    rf = pd.merge_asof(stocks.sort_values("datadate"), rf, direction="backward", on="datadate")[["datadate","RF"]]
    rf = rf.dropna()
    # Create a series
    rf = rf.drop_duplicates().set_index("datadate").squeeze()
    return rf


def get_equal_weighted_portfolio(tic_bins, stocks, rf):
    '''
    Back test a portfolio where are holdings are invested in equal proportions
        Parameters:
                tic_bins (DateFrame): data of each year and companies inside of each bin 
                stocks (DateFrame): data of each stock return over time
                rf (Series): data of risk free rate every month
            

        Returns:
                (DataFrame): Returns of each month for each bin
    '''
    equally_weighted_month_returns = []
    # For each year
    for i, row in tic_bins.iterrows():
        # Get the bins
        L, two, three, four, H, non = row["L"], row["2"], row["3"], row["4"], row["H"], row["Non R&D"]
        starting_date = row["date"]
        ending_date = starting_date + relativedelta(years=1)

        # Get market snapshot of that year
        market = stocks[(stocks["datadate"] >= starting_date) & (stocks["datadate"] < ending_date)].sort_values("datadate")

        return_row = [starting_date]
        binned_month_returns = []

        # For each bin
        for bin in [L, two, three, four, H, non]:

            # Find those stocks in that bin in the market snapshot
            bin_df = market[market['GVKEY'].isin(bin)]
            
            # Get the mean return per month
            monthly_avg_return = bin_df.groupby("datadate").mean()["trt1m"]
        
            # Calculate the excess return with respect to the risk free rate
            monthly_avg_return = monthly_avg_return.sub(rf).dropna()
            binned_month_returns.append(monthly_avg_return)
        
        equally_weighted_month_returns.append(pd.concat(binned_month_returns, axis=1))
    
    equal_weighted_monthly_returns_df = pd.concat(equally_weighted_month_returns)
    equal_weighted_monthly_returns_df.columns = ['L', '2', '3', '4', 'H', "Non R&D"]
    
    return equal_weighted_monthly_returns_df


def _get_weights(market, bin, date, unknown_market_cap_zero_weight):
    '''
    Helper function to construct value weighted portfolio; looks at past month market cap to determine the proportion invested of this month
        Parameters:
                market (DateFrame): Snapshot of the market in that year
                bin (DateFrame): the quintile that is currently being studied
                date (Series): current month being calculated; first of the month
            

        Returns:
                (dict): key=gvkey, value=weight for current month
    '''
    # Get the last month's market data
    last_month = date - relativedelta(days=1)
    last_month_stocks = market[(market["datadate"] == last_month) & (market["GVKEY"].isin(bin))]  
    
    # Set the default market cap if market cap not available last month
    if unknown_market_cap_zero_weight:
        default_market_cap = 0
    else:
        # Use a default market cap that can give equal weight with respect to number of stocks in bin like the equal weighted portfolio
        default_market_cap = last_month_stocks["Market Cap"].sum()/len(bin)
    
    # Find the stocks that have missing market caps last year
    missing_gvkeys = list(set(bin)-set(last_month_stocks["GVKEY"]))
    # For each missing stock, set the default market cap
    for missing_gvkey in missing_gvkeys:
        add_row = {'datadate': date - relativedelta(days=1), 'GVKEY': missing_gvkey, "LPERMNO": np.nan, "Market Cap": default_market_cap, "trt1m":np.nan}
        last_month_stocks = last_month_stocks.append(add_row, ignore_index = True)
    
    # Calculate the weight of each stock given their market cap
    last_month_stocks["weight"] = last_month_stocks["Market Cap"] / last_month_stocks["Market Cap"].sum()

    try:
        # Ensure that higher market cap stocks have higher weight
        assert (last_month_stocks["Market Cap"].sort_values().index == last_month_stocks["weight"].sort_values().index).all() == True
        # Ensure that all weights sum up to 1
        assert last_month_stocks["weight"].sum() > .999
    
    except AssertionError: 
        # if one of the check fail, run another check to see if equal values are misindexed
        list1 = last_month_stocks["Market Cap"].sort_values().index
        list2 = last_month_stocks["weight"].sort_values().index
        out_of_order = [(list1[i], list2[i]) for i in range(len(list1)) if list1[i]!= list2[i]]
        
        
        index = list(zip(*out_of_order))
        df1 = last_month_stocks.iloc[list(index[0])]
        df2 = last_month_stocks.iloc[list(index[1])]
        # See if stocks had equal market cap and weight and were misindexed
        if (df1[["Market Cap", "weight"]].sort_values("Market Cap").reset_index(drop=True).equals(df2[["Market Cap", "weight"]].sort_values("Market Cap").reset_index(drop=True))):
            pass

        else:
            # ERROR
            display(df1[["Market Cap", "weight"]].sort_values("Market Cap").reset_index(drop=True))
            display(df2[["Market Cap", "weight"]].sort_values("Market Cap").reset_index(drop=True))
            print(out_of_order)
            print(last_month_stocks["weight"].sum())

            raise AssertionError("Weighting Issue")
        
    return dict(zip(last_month_stocks["GVKEY"], last_month_stocks["weight"]))


# NOTE: Could have used existing library functions to compute weighted average of returns but because of missing data, sanity checks, and debugging, I have explicitly rewrote the method
def get_value_weighted_portfolio(tic_bins, stocks, rf, unknown_market_cap_zero_weight = False, debug = False):
    '''
    Back test a portfolio where are holdings are invested in proportions based on their previous month's market cap
        Parameters:
                tic_bins (DateFrame): data of each year and companies inside of each bin 
                stocks (DateFrame): data of each stock return over time
                rf (Series): data of risk free rate every month
                unknown_market_cap_zero_weight (bool): True if using zero weight for missing market cap; false if using equal weight
                debug (bool): prints debug statements regarding progress
            

        Returns:
                (DataFrame): Returns of each month for each bin
    '''
    
    return_rows = []
    value_weighted_month_returns = []
    with warnings.catch_warnings():
    
        warnings.simplefilter("ignore")
        for i, row in tic_bins.iterrows():
            L, two, three, four, H, non = row["L"], row["2"], row["3"], row["4"], row["H"], row["Non R&D"]
            starting_date = row["date"]
            if debug:
                print(i/len(tic_bins))
            ending_date = starting_date + relativedelta(years=1)


            market = stocks[(stocks["datadate"] >= starting_date - relativedelta(months=1)) & (stocks["datadate"] < ending_date)].sort_values("datadate")

            return_row = [starting_date]
            binned_month_returns = []
            for bin in [L, two, three, four, H, non]:
                current_month = starting_date
                
                return_dict = {}
                while current_month < ending_date:
                    return_weights = _get_weights(market, bin, current_month, unknown_market_cap_zero_weight)
                    this_month_return = market[(market["datadate"] == current_month + relativedelta(days=-1, months=1)) & (market["GVKEY"].isin(bin))]
                    this_month_return["weight"] = this_month_return['GVKEY'].map(return_weights)
                   
                    total_return = (this_month_return["trt1m"] * this_month_return["weight"]).sum()
                    
                    return_dict[current_month + relativedelta(days=-1, months=1)] = total_return
                    current_month += relativedelta(months=1)
                weighted_returns = pd.Series(return_dict)
                weighted_returns = weighted_returns.sub(rf).dropna()
                binned_month_returns.append(weighted_returns)
            
            value_weighted_month_returns.append(pd.concat(binned_month_returns, axis=1))
    
    value_weighted_month_returns_df = pd.concat(value_weighted_month_returns)
    value_weighted_month_returns_df.columns = ['L', '2', '3', '4', 'H', "Non R&D"]
    return value_weighted_month_returns_df

def get_results_table(monthly_returns):
    '''
    Create table that shows average monthly return for each time period
        Parameters:
               monthly_returns: returns of each month for each bin

        Returns:
                (DataFrame): average return of each month for each bin over time periods
    '''
    first_period = pd.Timestamp(year=1981, month=7, day=1)
    mid_period = pd.Timestamp(year=1999, month=12, day=31)
    last_period = pd.Timestamp(year=2012, month=12, day=31)
    extra_period = pd.Timestamp(year=2021, month=12, day=31)


    full_period = monthly_returns[(monthly_returns.index >= first_period) & (monthly_returns.index <= last_period)].mean()
    pre_2000 = monthly_returns[(monthly_returns.index >= first_period) & (monthly_returns.index <= mid_period)].mean()
    post_2000 = monthly_returns[(monthly_returns.index > mid_period) & (monthly_returns.index <= last_period)].mean()
    extra = monthly_returns[(monthly_returns.index >= first_period) & (monthly_returns.index <= extra_period)].mean()
    columns=['L', '2', '3', '4', 'H', 'Non R&D']


    results = pd.DataFrame({c: pd.Series(dtype="float") for c in columns})

    results = results.append(full_period,ignore_index=True)
    results = results.append(pre_2000,ignore_index=True)
    results = results.append(post_2000,ignore_index=True)
    results = results.append(extra,ignore_index=True)

    results.index = ["Full period", "Pre 2000", "Post 2000", "Full Through Dec 2021"]

    return results.round(3)

## Long-Short Functions

Below are functions that are used to test how well our strategy has done and if it produced a positive and significant alpha using regressions. There are also functions that compute the Sharpe ratio as well.

In [37]:

def get_fama_french_data(file = "data/F-F_Research_Data_Factors_monthly.CSV"):
    '''
    Get fama french factors
        Parameters:
                file (string): file path

        Returns:
                (DateFrame): factors data
    '''
    return pd.read_csv(file)

def prep_regression(ff_factors_monthly, monthly_returns, last_period = pd.Timestamp(year=2012, month=12, day=31)):
    '''
    Clean data and prepare it for regressions
        Parameters:
                ff_factors_monthly (DataFrame): factor data
                monthly_returns (DataFrame): monthly returns of each bin
                last_period (Timestamp): end date of regression

        Returns:
                (DateFrame): factors data cleaned
                (Series): High minus Low R&D returns
    '''
    # Beginning date of regression
    first_period = pd.Timestamp(year=1981, month=7, day=1)

    # Clean factor data
    ff_factors_monthly.rename({"Unnamed: 0":"Date"}, axis=1, inplace=True)
    ff_factors_monthly["Date"] = pd.to_datetime(ff_factors_monthly["Date"], format="%Y%m")

    # Get High R&D investment companies' return and minus them from Low R&D investment companies' return
    monthly_returns["HmL"] = monthly_returns["H"] - monthly_returns["L"]

    # Get desired time frame
    ff_factors_monthly = ff_factors_monthly[(ff_factors_monthly["Date"] >= first_period) &(ff_factors_monthly["Date"] <= last_period + relativedelta(days=1))]
    
    # Shift day back to match with monthly returns date
    ff_factors_monthly["Date"] = [i - relativedelta(days=1) for i in ff_factors_monthly["Date"]]
    
    # Get desired time frame
    HmL_rds = monthly_returns[(monthly_returns.index >= first_period )& (monthly_returns.index <= last_period)]["HmL"]

    return ff_factors_monthly, HmL_rds

def run_regression(ff_factors_monthly, HmL_rds, factors):
    '''
    Display regression results and summary
        Parameters:
                ff_factors_monthly (DataFrame): factor data
                HmL_rds (DataFrame): High minus Low R&D returns
                factors (list): factors that should be included in regression
    '''
    # Get features and target
    X = ff_factors_monthly[factors]
    y = HmL_rds.copy()


    # Match dates that are in both features and target
    remove_dates_X = list(set(ff_factors_monthly["Date"])- set(y.index))

    X_df = ff_factors_monthly[~ff_factors_monthly["Date"].isin(remove_dates_X)]

    # Ensure that all dates in target are in features
    assert list(set(y.index) - set(X_df["Date"])) == []
  

    # Run regression
    X = X_df[factors]
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    X_sm = sm.add_constant(X)
    model = sm.OLS(y,X_sm)
    results = model.fit()
    
    
    print("alpha: ", round(results.params[0], 2))
    print("t-stat: ", round(results.tvalues[0], 3))


def calculate_sharpe_ratio(y):
    '''
    Calculate the portfolio's sharpe ratio
        Parameters:
                y (Series): High minus low R&D returns
        Returns:
                (float): Calculated Sharpe Ratio
    '''
    rp = y.mean()
    std_rp = y.std()

    # Annual Sharpe Ratio
    sharpe_ratio = (rp)/std_rp * math.sqrt(12)
    return round(sharpe_ratio,3)


## Remove Large Cap Function

Below is a function that removes the 1000 large cap stocks from both the R&D firms and the non R&D firms as large cap stocks can significantly influence a value weighted portfolio more than small cap stocks which could significantly impact results. We can test results without these stock as well using this function.

In [26]:
def remove_top_1000(rd_firms, non_rd_firms):
    '''
    Remove top 1000 market cap firms from rd firms and non rd firms
        Parameters:
                rd_firms (DataFrame): All rd firms data
                non_rd_firms (DataFrame): All non rd firms data

        Returns:
                (DataFrame): rd firms data without largest 1000
                (DataFrame): non rd firms data without largest 1000
    '''
    # Sort by Market Cap for rd firms
    temp_1 = rd_firms.reset_index(drop=True).sort_values('Market Cap',ascending = False)
    # For each year get the top 1000 firms
    temp_2 = temp_1.groupby("year").head(1000).sort_values("year") 
    # Removes these top 1000 
    removed_top_rd = temp_1[~temp_1.index.isin(temp_2.index)]

    # Do the same for non rd firms
    temp_1 = non_rd_firms.reset_index(drop=True).sort_values('Market Cap',ascending = False)
    temp_2 = temp_1.groupby("year").head(1000).sort_values("year")  
    removed_top_non_rd = temp_1[~temp_1.index.isin(temp_2.index)]

    return removed_top_rd, removed_top_non_rd    





## Wrapper Functions

Functions below run all functions above in the correct sequential order

In [27]:
def get_data():
    
    '''
    Wraps all data gathering, cleaning, and splitting functions
        Returns:
                (DataFrame): rd firms data
                (DataFrame): non rd firms data
                (DataFrame): stock data
    '''
    print("Getting stock data")
    stocks = get_stocks_data()
    stocks = clean_stock_data(stocks)
    save_gvkeys(stocks)

    print("Getting Fundamentals")
    fundamentals = get_fundamentals()
    fundamentals = clean_fundamentals(fundamentals)

    print("Splitting Firms into RD and non RD")
    rd_firms, non_rd_firms = split_firms(fundamentals)
    rd_firms, non_rd_firms = get_market_cap(rd_firms, non_rd_firms, stocks)

    print("Calculating RD Capital / Market Cap")
    rd_firms = calculate_rd_capital(rd_firms)
    rd_firms = calculate_sorting_var(rd_firms)

    print("Finding most recent report each year")
    rd_firms, non_rd_firms = get_most_recent_report(rd_firms, non_rd_firms)

    print("Removing outliers")
    rd_firms = remove_sorting_var_outliers(rd_firms)

    return rd_firms, non_rd_firms, stocks

In [28]:
def get_regression_results(monthly_returns, last_period = pd.Timestamp(year=2012, month=12, day=31)):
    '''
    Wraps all regression functions
     Parameters:
                monthly_returns (DataFrame): Monthly returns for each bin
                last_period (Timestamp): end date of regression

        Returns:
                (Series): High minus Low R&D returns
    '''

    ff_factors_monthly = get_fama_french_data()
    ff_factors_monthly, HmL_rds = prep_regression(ff_factors_monthly, monthly_returns, last_period)
    
    print("\nCAPM Regression")
    run_regression(ff_factors_monthly, HmL_rds, ["Mkt-RF"])
    
    print("\nFama French Regression Results")
    run_regression(ff_factors_monthly, HmL_rds, ["Mkt-RF", "SMB", "HML"])
    
    return HmL_rds
    

In [39]:
def run_equal_portfolio(rd_firms, non_rd_firms, stocks):
    
    '''
    Wraps all equal portfolio functions
     Parameters:
                rd_firms (DataFrame): rd firms data
                non_rd_firms (DataFrame): non rd firms data
                stocks (DataFrame): stock data

    '''

    tic_bins = get_bins(rd_firms, non_rd_firms)
    rf = get_rf_rate_monthly(stocks)
    monthly_returns = get_equal_weighted_portfolio(tic_bins, stocks, rf)
    results = get_results_table(monthly_returns)
    
    print("\nAverage Monthly Return foe Equal Weighted Portfolio")
    display(results)
    
    print("\nRegression Results for 1981-2012")
    HmL_rds = get_regression_results(monthly_returns, last_period = pd.Timestamp(year=2012, month=12, day=31))
    
    print("\nSharpe Ratio (1981-2012):", calculate_sharpe_ratio(HmL_rds))

    
    print("\nRegression Results for 1981-2021")
    HmL_rds = get_regression_results(monthly_returns, last_period = pd.Timestamp(year=2021, month=12, day=31))
     
    print("\nSharpe Ratio (1981-2021):", calculate_sharpe_ratio(HmL_rds))

In [30]:
def run_value_portfolio(rd_firms, non_rd_firms, stocks, remove_top_1000_stocks = False, unknown_market_cap_zero_weight = True):
    '''
    Wraps all equal portfolio functions
     Parameters:
                rd_firms (DataFrame): rd firms data
                non_rd_firms (DataFrame): non rd firms data
                stocks (DataFrame): stock data
                remove_top_1000_stocks (boolean): True if need to remove largest 1000
                unknown_market_cap_zero_weight (boolean): True if setting 0 weight to unavailable market cap in previous month

    '''
    if remove_top_1000_stocks:
        rd_firms, non_rd_firms = remove_top_1000(rd_firms, non_rd_firms)

    tic_bins = get_bins(rd_firms, non_rd_firms)
    rf = get_rf_rate_monthly(stocks)
    monthly_returns = get_value_weighted_portfolio(tic_bins[tic_bins["date"] <= "2021-12-21"], stocks, rf, unknown_market_cap_zero_weight)
    results = get_results_table(monthly_returns)
    
    if remove_top_1000_stocks:
        print("\nAverage Monthly Return for Value Weighted Portfolio without Top 1000")
    else:
        print("\nAverage Monthly Return for Value Weighted Portfolio")
    display(results)
    
    print("\nRegression Results for 1981-2012")
    HmL_rds = get_regression_results(monthly_returns, last_period = pd.Timestamp(year=2012, month=12, day=31))
    
    print("\nSharpe Ratio (1981-2012):", calculate_sharpe_ratio(HmL_rds))

    
    print("\nRegression Results for 1981-2021")
    HmL_rds = get_regression_results(monthly_returns, last_period = pd.Timestamp(year=2021, month=12, day=31))
     
    print("\nSharpe Ratio (1981-2021):", calculate_sharpe_ratio(HmL_rds))

## Run all test

On current machine, takes a little over an hour to run all tests due to iterative weight calculation

In [31]:
def run():
    '''
    Main function to run all tests and results
    '''
    with warnings.catch_warnings():
    
        warnings.simplefilter("ignore")
        
        print("Gathering Data...")
        rd_firms, non_rd_firms, stocks = get_data()
        
        print("\nCreating Equal Weighted Portfolio...")
        run_equal_portfolio(rd_firms, non_rd_firms, stocks)
        
        unknown_market_cap_zero_weight = True

        print("\n\nCreating Value Weighted Portfolio...")
        run_value_portfolio(rd_firms, non_rd_firms, stocks, unknown_market_cap_zero_weight=unknown_market_cap_zero_weight)

        print("\n\nCreating Value Weighted Portfolio without Top 1000...")
        run_value_portfolio(rd_firms, non_rd_firms, stocks, remove_top_1000_stocks=True, unknown_market_cap_zero_weight=unknown_market_cap_zero_weight)

        print("Done.")
    

In [40]:
run()

Gathering Data...
Getting stock data
Getting Fundamentals
Splitting Firms into RD and non RD
Calculating RD Capital / Market Cap
Finding most recent report each year
Removing outliers

Creating Equal Weighted Portfolio...

Average Monthly Return foe Equal Weighted Portfolio


,L,2,3,4,H,Non R&D
Full period,0.634,0.717,0.933,1.235,1.768,0.694
Pre 2000,0.605,0.677,0.958,1.261,1.817,0.606
Post 2000,0.677,0.774,0.897,1.197,1.697,0.820
Full Through Dec 2021,0.713,0.831,1.050,1.276,1.709,0.736



Regression Results for 1981-2012

CAPM Regression
alpha:  1.09
t-stat:  4.243

Fama French Regression Results
alpha:  1.01
t-stat:  3.892

Sharpe Ratio (1981-2012): 0.792

Regression Results for 1981-2021

CAPM Regression
alpha:  0.95
t-stat:  4.257

Fama French Regression Results
alpha:  0.93
t-stat:  4.142

Sharpe Ratio (1981-2021): 0.709


Creating Value Weighted Portfolio...

Average Monthly Return for Value Weighted Portfolio


,L,2,3,4,H,Non R&D
Full period,0.475,0.625,0.633,0.923,0.835,0.593
Pre 2000,0.678,1.020,0.901,1.246,1.226,0.764
Post 2000,0.186,0.062,0.253,0.465,0.279,0.350
Full Through Dec 2021,0.536,0.811,0.794,1.016,0.941,0.663



Regression Results for 1981-2012

CAPM Regression
alpha:  0.32
t-stat:  1.25

Fama French Regression Results
alpha:  0.2
t-stat:  0.784

Sharpe Ratio (1981-2012): 0.254

Regression Results for 1981-2021

CAPM Regression
alpha:  0.37
t-stat:  1.743

Fama French Regression Results
alpha:  0.33
t-stat:  1.543

Sharpe Ratio (1981-2021): 0.302


Creating Value Weighted Portfolio without Top 1000...

Average Monthly Return for Value Weighted Portfolio without Top 1000


,L,2,3,4,H,Non R&D
Full period,0.444,0.752,1.069,0.844,1.680,0.341
Pre 2000,0.356,0.809,1.247,0.932,1.660,0.087
Post 2000,0.558,0.679,0.838,0.731,1.706,0.668
Full Through Dec 2021,0.586,0.777,1.061,0.906,1.376,0.452



Regression Results for 1981-2012

CAPM Regression
alpha:  1.16
t-stat:  3.186

Fama French Regression Results
alpha:  1.1
t-stat:  2.993

Sharpe Ratio (1981-2012): 0.627

Regression Results for 1981-2021

CAPM Regression
alpha:  0.75
t-stat:  2.363

Fama French Regression Results
alpha:  0.74
t-stat:  2.327

Sharpe Ratio (1981-2021): 0.405
Done.
